In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Environment, Model, ManagedOnlineEndpoint, ManagedOnlineDeployment, CodeConfiguration
import uuid

# Azure ML workspace details
subscription_id = "14b6d2dd-fe2d-4c59-bc9a-ee949d79cfef"
resource_group = "usecasehws1kdur"
workspace_name = "usecasetesthws1kdur"

# Initialize MLClient
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace_name
)

# Unique endpoint name
endpoint_name = f"dummy-classifier-{uuid.uuid4().hex[:8]}"

# Create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key"
)
print(f"Creating endpoint {endpoint_name}...")
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

# Create environment configuration
env = Environment(
    name="multilabel-env",
    description="Environment for multilabel classification model",
    conda_file="environment.yaml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest"
)
ml_client.environments.create_or_update(env)

# Deployment configuration
deployment = ManagedOnlineDeployment(
    name="default",
    endpoint_name=endpoint_name,
    model="azureml:multilabel_classifier:16", # change the version
    environment=env,
    # instance_type="Standard_DS3_v2",
    code_configuration=CodeConfiguration(code=".", scoring_script='score.py'),
    environment_variables={
        "AZURE_SUBSCRIPTION_ID": subscription_id,
        "AZURE_RESOURCE_GROUP": resource_group,
        "AZURE_WORKSPACE_NAME": workspace_name
    }
)
print(f"Creating deployment...")
ml_client.online_deployments.begin_create_or_update(deployment).result()

# Set the deployment as default
endpoint.traffic = {"default": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Deployment completed. Endpoint {endpoint_name} is ready.")

Creating endpoint dummy-classifier-0b5c7cf2...
.........

Check: endpoint dummy-classifier-0b5c7cf2 exists
Uploading deploy (0.01 MBs): 100%|██████████| 11367/11367 [00:00<00:00, 256632.40it/s]


